# 뉴스 헤드라인 분류하기 (로컬 버전)

> 이 노트북은 SageMaker Notebook Instance 및 conda_pytorch_p310 및 SageMaker Studio의 `Python 3 (PyTorch 1.13 Python 3.9 CPU Optimized)` 커널에서 잘 작동합니다.

이 예제에서는 커스텀 스크립트와 [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) 프레임워크를 사용하여 뉴스 헤드라인 분류기 모델을 훈련할 것입니다.

이 "로컬" 노트북은 노트북 자체에서 모델을 훈련하고 테스트하는 방법을 보여줄 것이며, 동반 ["SageMaker" 노트북](Headline%20Classifier%20SageMaker.ipynb)은 컨테이너화된 SageMaker 훈련 작업과 엔드포인트 배포를 사용하여 동일한 프로세스를 반복할 것입니다.

Hugging Face를 처음 접하는 경우 [Transformers 빠른 투어](https://huggingface.co/docs/transformers/quicktour)를 읽거나 다음 (1시간) 소개 동영상을 시청하는 것이 좋습니다:

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/pYqjCzoyWyo" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## 설치 및 설정

위에서 지정한 PyTorch SageMaker 커널은 필요한 대부분의 라이브러리를 가지고 있지만 모든 것을 가지고 있지는 않습니다. 먼저, HF transformers/datasets의 적절한 버전과 나중에 대화형 분류기 위젯을 구동하기 위한 IPyWidgets를 설치해야 합니다:

> ⚠️ **참고:** 이 셀을 먼저 실행하면 노트북 커널을 다시 시작할 필요가 없습니다. 하지만 이미 `import`한 것이 있다면 이러한 설치가 적용되도록 위의 툴바에서 원형 '커널 다시 시작' 버튼을 클릭해야 합니다.

아래 출력에서 pip의 *경고*는 무시할 수 있지만 *오류*는 보이지 않아야 합니다.

In [2]:
%pip install datasets transformers==4.26

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.17
    Uninstalling multiprocess-0.70.17:
      Successfully uninstalled multiprocess-0.70.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.8 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, but you have multiprocess 0.70.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%sh
pip install --upgrade jupyter jupyterlab
pip install --upgrade ipywidgets

  Using cached jupyterlab-4.3.3-py3-none-any.whl.metadata (16 kB)


설치가 완료되면 나머지 노트북에서 사용할 라이브러리와 Python 내장 기능을 로드합니다.

[%autoreload magic](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html)은 로컬 .py 파일로 작업할 때 유용합니다. 각 셀 실행 시 라이브러리를 다시 로드하면 노트북 커널을 다시 시작하지 않고도 로컬로 편집/업데이트된 스크립트를 사용할 수 있기 때문입니다.

In [4]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os  # Operating system utils e.g. file paths

# External Dependencies:
import datasets  # Hugging Face data loading utilities
import ipywidgets as widgets  # Interactive prediction widget
import pandas as pd  # Utilities for working with data tables (dataframes)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import transformers  # Hugging Face Transformers framework

local_dir = "data"

## 데이터셋 준비하기

이 예제는 [Registry of Open Data on AWS](https://registry.opendata.aws/fast-ai-nlp/) 공개 리포지토리에서 **FastAi AG News** 데이터셋을 다운로드합니다. 이 데이터셋에는 뉴스 헤드라인과 해당 주제 클래스가 포함된 테이블이 들어 있습니다.

In [5]:
%%time
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner

# Push data partitions to separate subfolders, which is useful for local script debugging later
os.renames(f"{local_dir}/test.csv", f"{local_dir}/test/test.csv")
os.renames(f"{local_dir}/train.csv", f"{local_dir}/train/train.csv")
print("Done!")

download: s3://fast-ai-nlp/ag_news_csv.tgz to data/ag_news_csv.tgz
Done!
CPU times: user 17 ms, sys: 39.5 ms, total: 56.5 ms
Wall time: 1.44 s


데이터를 다운로드하고 추출한 후 아래와 같이 일부 예제를 탐색할 수 있습니다:

In [6]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)

# Make the (1-indexed) category classes more readable:
class_names = ["Other", "World", "Sports", "Business", "Sci/Tech"]
idx2label = {ix: name for ix, name in enumerate(class_names)}
label2idx = {name: ix for ix, name in enumerate(class_names)}

df = df.replace({"CATEGORY": idx2label})
df.head()

,CATEGORY,TITLE,CONTENT
86110,Sci/Tech,Oracle to drop PeopleSoft suit if tender fails,Oracle Corp. notified Delaware's Court of Chan...
74390,Sci/Tech,"NTT DoCoMo, IBM, Intel team to secure mobile d...",With an eye towards making mobile devices and ...
77491,Sci/Tech,Election Is Crunch Time for U.S. Secret Service,With just days to go before the U.S. president...
27497,Sports,Former Celtic striker Larsson on Barcelona bench,Henrik Larsson was left on the bench by Barcel...
47492,World,Four Suicides Linked to Child Porn Probe (AP),AP - The government will press on with a child...


이 연습에서 우리는 **다음만 사용할 것입니다**:

- 뉴스 기사의 **제목**(헤드라인)을 입력으로 사용합니다.
- **카테고리**를 예측할 대상 변수로 사용합니다.

이 데이터셋에는 아래와 같이 4개의 고르게 분포된 주제 클래스가 포함되어 있습니다.

> ℹ️ **'기타'에 대해:** 원시 데이터셋은 1-4의 숫자로 카테고리를 나타내고, 모델은 0부터 시작하는 숫자를 기대하므로, 데이터 준비를 단순하게 유지하고 클래스의 추가적이고 혼란스러운 숫자 표현을 도입하지 않기 위해 사용되지 않은 '기타' 클래스를 삽입했습니다.

In [7]:
df["CATEGORY"].value_counts()

Sci/Tech    30000
Sports      30000
World       30000
Business    30000
Name: CATEGORY, dtype: int64

## 훈련 매개변수 정의하기

우리는 [Hugging Face Hub](https://huggingface.co/models)에서 (비교적 작은) 사전 훈련된 모델을 미세 조정할 것이며, 낮은 수준의 훈련 루프를 처음부터 작성하는 대신 높은 수준의 [Trainer API](https://huggingface.co/docs/transformers/main_classes/trainer)를 사용할 것입니다.

아래에서 훈련을 위한 기본 매개변수를 설정할 것입니다.

> 🏎️ 이 노트북 내 예제의 경우 기본적으로 **저비용, CPU 전용 컴퓨팅**을 사용할 것입니다. 우리가 훈련하는 모델은 현대 LLM 표준에 비해 "작은" 모델이지만, 합리적인 시간 내에 완료될 수 있도록 훈련을 매우 일찍 중단해야 할 것입니다.
>
> 결과적으로 생성된 모델은 매우 부족하게 훈련되며, 동일한 아키텍처가 궁극적으로 얻을 수 있는 것보다 훨씬 덜 정확할 것입니다.

In [8]:
model_id = "amazon/bort"  # ID of the pre-trained model to start from

training_args = transformers.TrainingArguments(
    output_dir=f"{local_dir}/model",  # Where to save trained model snapshots
    #logging_dir=f"{local_dir}/local-logs",  # Optionally, save logs too
    max_steps=500,  # Maximum number of training steps to run
    num_train_epochs=3,  # Maximum number of times to loop through the training data
    per_device_train_batch_size=16,  # Examples per mini-batch for training
    per_device_eval_batch_size=32,  # Examples per mini-batch for validation
    evaluation_strategy="steps",  # Run validation every N 'steps' instead of every 'epoch'
    eval_steps=100,  # Number of training steps between validation runs
    save_strategy="steps",  # Must be same as evaluation_strategy when load_best_model_at_end=True
    load_best_model_at_end=True,  # If current model at end is not the best, load the best
    metric_for_best_model="f1",  # Use F1 score for judging which model is 'best'
    learning_rate=5e-5,  # Initial learning rate (decays over time by default)
    warmup_steps=100,  # Number of steps to gradually increase the learning rate from the start
)

## 지표 정의하기

여기서는 모델이 검증될 때마다 실행될 [콜백 함수](https://huggingface.co/docs/transformers/main_classes/callback)를 설정하여 훈련된 모델의 품질을 측정하고자 하는 방법을 정의할 것입니다.

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

## 모델 훈련 및 검증하기

이 섹션에서는 기본 모델과 데이터셋을 로드하고 실제 훈련 및 검증 프로세스를 실행할 것입니다.

먼저, 주어진 `model_id`에 대한 사전 훈련된 모델과 함께 제공되는 [토크나이저](https://huggingface.co/docs/transformers/main_classes/tokenizer)를 로드해야 합니다. 이는 Hugging Face Hub에서 자동으로 다운로드됩니다.

모델을 설정하는 과정에서 미세 조정할 주제 클래스의 수를 지정하고 사람이 읽을 수 있는 이름을 설정해야 합니다:

In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(class_names))
model.config.label2id = label2idx
model.config.id2label = idx2label

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at amazon/bort were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSeque

데이터셋에 이미 제공된 원시 `train.csv` 및 `test.csv` 파일을 훈련의 입력으로 사용할 것이지만, 먼저 일부 사전 처리를 설정해야 합니다:

- CSV에 열 헤더가 없기 때문에 `column_names`를 수동으로 지정해야 합니다.
- `tokenizer`는 모델이 기대하는 (숫자) 입력으로 원시 텍스트를 변환합니다. 여기에는 모델에서 지원하는 최대 길이로 긴 헤드라인을 자르는 작업도 포함됩니다.

In [11]:
def preprocess(batch):
    """Tokenize and pre-process raw examples for training/validation"""
    result = tokenizer(batch["title"], truncation=True)
    result["label"] = batch["category"]
    return result


# Load the raw datasets:
raw_train_dataset = datasets.load_dataset(
    "csv",
    data_files=os.path.join(local_dir, "train", "train.csv"),
    column_names=["category", "title", "content"],
    split=datasets.Split.ALL,
)
raw_test_dataset = datasets.load_dataset(
    "csv",
    data_files=os.path.join(local_dir, "test", "test.csv"),
    column_names=["category", "title", "content"],
    split=datasets.Split.ALL,
)

# Run the tokenization/pre-processing, keeping only the output fields from preprocess()
train_dataset = raw_train_dataset.map(
    preprocess, batched=True, batch_size=1000, remove_columns=raw_train_dataset.column_names
)
test_dataset = raw_test_dataset.map(
    preprocess, batched=True, batch_size=1000, remove_columns=raw_test_dataset.column_names
)

매개변수와 사전 처리된 데이터가 로드되었으므로 이제 모델을 훈련하고 평가할 준비가 되었습니다.

> ⏰ **참고:** 기본 `ml.t3.medium` (2 vCPU + 4 GiB RAM) Studio 인스턴스 유형에서 이 프로세스는 완료되기까지 약 20분이 소요됩니다.
>
> 기다리는 동안 [SageMaker 노트북](Headline%20Classifier%20SageMaker.ipynb)으로 이동하여 이 프로세스가 SageMaker 훈련 작업으로 마이그레이션될 때 어떻게 다를지 탐색할 수 있습니다.

In [12]:
%%time

# create Trainer instance
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# train model
trainer.train()

# evaluate model
eval_result = trainer.evaluate(eval_dataset=test_dataset)

max_steps is given, it will override any value given in num_train_epochs
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 120000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 76162053
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,No log,1.486111,0.250000,0.250000,0.250000,0.250000
200,No log,1.465181,0.271316,0.271316,0.271316,0.271316
300,No log,1.368726,0.277632,0.277632,0.277632,0.277632
400,No log,1.260783,0.378816,0.378816,0.378816,0.378816
500,1.378200,1.217790,0.428289,0.428289,0.428289,0.428289


***** Running Evaluation *****
  Num examples = 7600
  Batch size = 32
***** Running Evaluation *****
  Num examples = 7600
  Batch size = 32
***** Running Evaluation *****
  Num examples = 7600
  Batch size = 32
***** Running Evaluation *****
  Num examples = 7600
  Batch size = 32
***** Running Evaluation *****
  Num examples = 7600
  Batch size = 32
Saving model checkpoint to data/model/checkpoint-500
Configuration saved in data/model/checkpoint-500/config.json
Model weights saved in data/model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in data/model/checkpoint-500/tokenizer_config.json
Special tokens file saved in data/model/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from data/model/checkpoint-500 (score: 0.42828947368421055).
***** Running Evaluation *****
  Num examples = 7600
  Batch size = 32


CPU times: user 22.5 s, sys: 2.21 s, total: 24.7 s
Wall time: 25.4 s


지표에서 볼 수 있듯이 여기서 훈련된 모델은 아마도 매우 정확하지 않을 것이며, 훈련이 종료된 시점에서도 정확도는 여전히 빠르게 증가할 것입니다.

## 모델을 추론에 사용하기

모델이 훈련되면 새 데이터에 대한 추론에 사용할 준비가 되었습니다.

여기서 모델은 이미 훈련 프로세스에서 메모리에 로드되어 있으므로 쉽게 사용할 수 있도록 [Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)으로 감쌀 수 있습니다.

아래 셀은 사용자가 자신의 뉴스 헤드라인을 입력하고 모델이 실시간으로 분류할 수 있는 대화형 위젯을 생성합니다:

In [13]:
import torch

# GPU 사용 가능 여부에 따라 device 인덱스 지정
device = 0 if torch.cuda.is_available() else -1

# 파이프라인 생성 시 device 지정
pipe = transformers.pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device
)

# pipe = transformers.pipeline(
#     task="text-classification",
#     model=model,
#     tokenizer=tokenizer,
# )


def classify(text: str) -> dict:
    """Classify a headline and print the results"""
    print(pipe(text)[0])


# Either try out the interactive widget:
interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

interactive(children=(Text(value='The markets were bullish after news of the merger', continuous_update=False,…

Alternatively, you can call the pipeline direct from code:

In [14]:
classify("Retailers are expanding after the recent economic growth")

{'label': 'Sports', 'score': 0.3337773382663727}


## 검토

이 노트북에서는 일반 Jupyter 환경에서 Hugging Face transformers를 사용하여 텍스트 분류 모델을 훈련하는 방법을 보여주었습니다.

기본 노트북 컴퓨팅 인프라(`ml.t3.medium`)가 상당히 작았기 때문에 훈련에 오랜 시간이 걸렸고 결과를 탐색할 기회를 얻기 위해 일찍 중단해야 했습니다.

- 더 나은 모델을 훈련하기 위해 훈련 에포크/스텝 컷오프를 확장할 수 있지만, 그러면 프로세스가 더 오래 걸릴 것입니다.
- Studio 노트북을 더 높은 리소스 인스턴스(아마도 GPU를 사용하여)로 전환할 수 있지만, 그러면 모델을 실제로 훈련하지 않는 시간(예: 데이터 탐색 또는 평가 중)에 추가 리소스가 유휴 상태일 수 있습니다.
- 또한 훈련 프로세스에서 시도한 다른 매개변수를 추적하기 위해 실험을 수동으로 기록해야 합니다.

다음으로, [SageMaker 노트북](Headline%20Classifier%20SageMaker.ipynb)으로 이동하여 SageMaker 훈련 작업과 엔드포인트 배포가 더 빠른 훈련과 자동 메타데이터 추적을 위해 온디맨드 컴퓨팅을 활용하는 데 어떻게 도움이 되는지 보여줄 것입니다. 이 경우 필요한 것에 대해서만 비용을 지불하게 됩니다.